In [20]:
# Import packages
import os
import glob
import shutil
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# Reproducibility
np.random.seed(478)
tf.random.set_seed(478)

In [25]:
# Read in data and prepare for training
train = pd.read_csv('./Data/train.csv')
test = pd.read_csv('./Data/test.csv')

idx = range(25200)

# Manual creation of pd dataframe training and validation sets
x = train.iloc[train.index.isin(idx)].drop('label', axis = 1)
y = train['label'].iloc[train.index.isin(idx)]
valid = train[~train.index.isin(idx)]

# Experimental conversion to tensors
testx = tf.convert_to_tensor(x)
testy = tf.convert_to_tensor(y)

In [24]:
BASE_DIR = './Data'
classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

for cl in classes:
  img_path = os.path.join(BASE_DIR, cl)
  images = glob.glob(img_path + '/*.jpg')
  print("{}: {} Images".format(cl, len(images)))
  train, val = images[:round(len(images)*0.7)], images[round(len(images)*0.7):]

  for t in train:
    if not os.path.exists(os.path.join(BASE_DIR, 'train', cl)):
      os.makedirs(os.path.join(BASE_DIR, 'train', cl))
    shutil.move(t, os.path.join(BASE_DIR, 'train', cl))

  for v in val:
    if not os.path.exists(os.path.join(BASE_DIR, 'val', cl)):
      os.makedirs(os.path.join(BASE_DIR, 'val', cl))
    shutil.move(v, os.path.join(BASE_DIR, 'val', cl))

train_dir = os.path.join(BASE_DIR, 'train')
val_dir = os.path.join(BASE_DIR, 'val')

image_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/.255)
train_data = image_gen.flow_from_directory(directory = train_dir, shuffle = True, target_size = (128, 128))
valid_data = image_gen.flow_from_directory(directory = val_dir, shuffle = True, target_size = (128, 128))

0: 0 Images
1: 0 Images
2: 0 Images
3: 0 Images
4: 0 Images
5: 0 Images
6: 0 Images
7: 0 Images
8: 0 Images
9: 0 Images


FileNotFoundError: [Errno 2] No such file or directory: './Data/train'

In [11]:
# Define model architecture
model = keras.Sequential([
    layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu', input_shape = [28, 28, 1]),
    layers.MaxPool2D(),

    layers.Conv2D(filters = 64, kernel_size = 3, activation = 'relu'),
    layers.MaxPool2D(),

    layers.Flatten(),
    layers.Dense(units = 10, activation = tf.nn.softmax)
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                16010     
Total params: 34,826
Trainable params: 34,826
Non-trainable params: 0
__________________________________________________

In [26]:
# Compile and fit model
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

trained = model.fit(
    testx,
    testy,
    epochs = 20
)

Epoch 1/20


ValueError: in user code:

    /Users/akiehl/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /Users/akiehl/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Users/akiehl/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Users/akiehl/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Users/akiehl/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /Users/akiehl/opt/anaconda3/lib/python3